In [16]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters \
  pinecone-text    

In [17]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
from pinecone.grpc import PineconeGRPC as Pinecone
#from langchain.vectorstores import PineconeVectorStore  
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [18]:
#  import nltk
#  nltk.download('punkt')

In [19]:
import os
from dotenv import load_dotenv

load_dotenv()
# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')


# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

In [59]:
directory = 'data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

docs = load_docs(directory)

In [21]:
#docs

In [22]:
#print(len(docs))

In [23]:
# print(docs[0].page_content[0:100])
# print(docs[0].metadata)

In [24]:

os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x32464f920>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x32478a270>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [25]:
import getpass
import os
import time
from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

In [26]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[(vector_id, vector, metadata)])  # Make sure to include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index)


In [27]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index, tenant_id):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source, tenantId, and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "tenantId": tenantId,  # Add tenant-specific ID
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector_id, "values": vector, "metadata": metadata}])  # Include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}, Tenant ID: {tenantId}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Example of embedding generation and Pinecone index setup
# # embeddings and index should already be initialized
# tenantId = "example_tenantId"  # Add your tenantId here

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index, tenantId)


In [28]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector_id, "values": vector, "metadata": metadata}])  # Include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Example of embedding generation and Pinecone index setup
# # embeddings and index should already be initialized

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index)


In [29]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader


# directory = 'data'
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to upload document chunks with ID prefixes
# def upload_documents(directory, embeddings, index):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate the document ID prefix
#             doc_id_prefix = f"{filename.split('.')[0]}"

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source and content preview)
#                 metadata = {
#                     "source": filename,
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Call the function to upload documents from the directory
# upload_documents(directory, embeddings, index)

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids


In [30]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# import random
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter


# directory = 'data'
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a random 8-digit ID
# def generate_random_id():
#     return str(random.randint(10000000, 99999999))

# # Function to upload document chunks with a random 8-digit ID prefix
# def upload_documents(directory, embeddings, index):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate a random 8-digit document ID prefix
#             doc_id_prefix = generate_random_id()

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source and content preview)
#                 metadata = {
#                     "source": filename,
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids

# # Call the function to upload documents from the directory
# upload_documents(directory, embeddings, index)


## Prefix

In [31]:
import os
import concurrent.futures
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


directory = 'data'
chunk_size = 1000  # Adjust based on your use case
chunk_overlap = 100  # Adjust based on your use case
batch_size = 50  # Number of chunks per batch for upsert to Pinecone


# Function to read text from different file types
def read_document(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return "\n".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        return None  # Unsupported file type


# Function to process and upsert a single document
def process_and_upload_document(file_path, embeddings, index, tenantId):
    document_text = read_document(file_path)

    if document_text:
        # Generate the document ID prefix
        doc_id_prefix = f"{os.path.basename(file_path).split('.')[0]}"
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Split the document into chunks
        chunks = text_splitter.split_text(document_text)
        vectors = []
        
        # Create vectors for each chunk
        for i, chunk in enumerate(chunks):
            chunk_id = f"{doc_id_prefix}#chunk{i+1}"
            vector = embeddings.embed_documents([chunk])[0]
            metadata = {
                "filename": os.path.basename(file_path),
                "tenantId": tenantId,  # Add tenant_ID to metadata
            }
            vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

            # Batch upsert after processing `batch_size` chunks
            if len(vectors) >= batch_size:
                index.upsert(vectors=vectors, namespace="")
                vectors.clear()  # Clear the batch after uploading

        # Upsert any remaining vectors
        if vectors:
            index.upsert(vectors=vectors, namespace="")
        
        print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")
    else:
        print(f"Skipping unsupported or empty file: {file_path}")


# Function to upload documents in parallel
def upload_documents_concurrently(directory, embeddings, index, tenantId):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            futures.append(executor.submit(process_and_upload_document, file_path, embeddings, index, tenantId))
        
        # Wait for all threads to complete
        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("All documents uploaded successfully!")


# Call the function to upload documents from the directory concurrently
tenantId = "your_tenantId_here"  # Add tenant ID here or pass dynamically
upload_documents_concurrently(directory, embeddings, index, tenantId)


# Function to list all chunks of a document by prefix
def list_document_chunks(doc_id_prefix):
    chunk_ids = []
    for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
        chunk_ids.extend(ids)
    return chunk_ids


Skipping unsupported or empty file: data/.DS_Store
Uploaded Document - Prefix: world war, Total Chunks: 15
All documents uploaded successfully!


In [32]:
import os
import concurrent.futures
import uuid  # For generating unique IDs
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

directory = 'data'
chunk_size = 1000  # Adjust based on your use case
chunk_overlap = 100  # Adjust based on your use case
batch_size = 50  # Number of chunks per batch for upsert to Pinecone

# Function to read text from different file types
def read_document(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return "\n".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        return None  # Unsupported file type

# Function to process and upsert a single document
def process_and_upload_document(file_path, embeddings, index, tenantId):
    document_text = read_document(file_path)

    if document_text:
        # Generate a unique ID for the entire document
        embedded_id = str(uuid.uuid4())
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Split the document into chunks
        chunks = text_splitter.split_text(document_text)
        vectors = []
        
        # Create vectors for each chunk using the same embedded_id
        for chunk in chunks:
            vector = embeddings.embed_documents([chunk])[0]
            metadata = {
                "filename": os.path.basename(file_path),
                "tenantId": tenantId,  # Add tenant_ID to metadata
            }
            vectors.append({"id": embedded_id, "values": vector, "metadata": metadata})

            # Batch upsert after processing `batch_size` chunks
            if len(vectors) >= batch_size:
                index.upsert(vectors=vectors, namespace="")
                vectors.clear()  # Clear the batch after uploading

        # Upsert any remaining vectors
        if vectors:
            index.upsert(vectors=vectors, namespace="")
        
        # Print the filename, tenant ID, and the shared embedded ID for the document once after all chunks are uploaded
        print(f"fileName: {os.path.basename(file_path)}, tenantId: {tenantId}, embeddedId: {embedded_id}")
        
        # Return the total number of chunks for this document
        return len(chunks)

    else:
        print(f"Skipping unsupported or empty file: {file_path}")
        return 0  # Return 0 if no valid document was processed

def upload_documents_concurrently(directory, embeddings, index, tenantId):
    total_chunks = 0  # Counter for total chunks across all documents

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for filename in os.listdir(directory):
            # Skip unsupported or empty files
            if filename == ".DS_Store":
                continue

            file_path = os.path.join(directory, filename)
            futures.append(executor.submit(process_and_upload_document, file_path, embeddings, index, tenantId))
        
        # Wait for all threads to complete
        for future in concurrent.futures.as_completed(futures):
            total_chunks += future.result()  # Aggregate the total chunks

    print(f"All documents uploaded successfully! Total number of chunks inserted to Pinecone: {total_chunks}")

# Call the function to upload documents from the directory concurrently
tenantId = "Vitafy"  # Add tenant ID here or pass dynamically
upload_documents_concurrently(directory, embeddings, index, tenantId)

# Function to list all chunks of a document by embedded ID
def list_document_chunks(embeddedId):
    chunk_ids = []
    for ids in index.list(prefix=f"{embeddedId}", namespace=""):
        chunk_ids.extend(ids)
    return chunk_ids


fileName: world war.pdf, tenantId: Vitafy, embeddedId: 8a981444-0737-4449-a273-8288ba9309eb
All documents uploaded successfully! Total number of chunks inserted to Pinecone: 15


In [33]:
import os
import concurrent.futures


# Function to list all chunks of a document by prefix
def list_document_chunks(doc_id_prefix):
    chunk_ids = []
    # Assuming index.list supports pagination or has a mechanism to retrieve chunks efficiently
    for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
        chunk_ids.extend(ids)
    return chunk_ids


# Function to delete all chunks of a document by prefix in parallel
def delete_document_by_prefix(doc_id_prefix, batch_size=100):
    chunk_ids = list_document_chunks(doc_id_prefix)
    
    if chunk_ids:
        def delete_chunks_batch(batch):
            index.delete(ids=batch, namespace="")
        
        # Batch the chunk deletions to reduce overhead
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Split chunk_ids into batches of size `batch_size`
            batches = [chunk_ids[i:i + batch_size] for i in range(0, len(chunk_ids), batch_size)]
            futures = [executor.submit(delete_chunks_batch, batch) for batch in batches]

            # Wait for all batch deletions to complete
            for future in concurrent.futures.as_completed(futures):
                future.result()

        print(f"Deleted {len(chunk_ids)} chunks for document: {doc_id_prefix}")
    else:
        print(f"No chunks found for document: {doc_id_prefix}")


# Function to upload documents in parallel (unchanged)
def upload_documents_concurrently(directory, embeddings, index, tenant_ID):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            futures.append(executor.submit(process_and_upload_document, file_path, embeddings, index, tenantId))
        
        # Wait for all threads to complete
        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("All documents uploaded successfully!")


# Example usage
doc_id_prefix = "Settings Module"  # Replace with your actual document name

# Delete all chunks for a document in parallel
delete_document_by_prefix(doc_id_prefix)


No chunks found for document: Settings Module


In [34]:
# import os
# import uuid
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # directory = 'data'
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # # Function to load documents
# # def load_docs(directory):
# #     loader = DirectoryLoader(directory)
# #     docs = loader.load()  # Load documents using the DirectoryLoader
# #     return docs

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to upload document chunks with a random 14-digit ID prefix
# def upload_documents(directory, embeddings, index, tenant_ID):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate a random 14-digit document ID prefix
#             doc_id_prefix = generate_14_digit_random_id()

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source, tenant_ID, and content preview)
#                 metadata = {
#                     "source": filename,
#                     "tenant_ID": tenant_ID,  # Store the tenant ID as metadata
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             #print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids

# # Example call to upload documents from the directory
# tenant_ID = "tenant1234"  # Example tenant ID
# upload_documents(directory, embeddings, index, tenant_ID)


In [35]:
# import os
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # Define constants
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to process each document
# def process_document(filename, directory, embeddings, index, tenant_ID):
#     file_path = os.path.join(directory, filename)
#     document_text = read_document(file_path)
    
#     if document_text:
#         # Generate a random 14-digit document ID prefix
#         doc_id_prefix = generate_14_digit_random_id()

#         # Split the document into chunks
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         chunks = text_splitter.split_text(document_text)

#         # Generate embeddings for chunks in batches
#         vectors = []
#         for i, chunk in enumerate(chunks):
#             chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#             vectors.append({"id": chunk_id, "content": chunk})

#         # Generate embeddings for all chunks at once
#         if vectors:
#             content_list = [vector["content"] for vector in vectors]
#             embedded_vectors = embeddings.embed_documents(content_list)

#             # Prepare the vectors for upsert
#             for vector, embedding in zip(vectors, embedded_vectors):
#                 vector["values"] = embedding
#                 vector["metadata"] = {
#                     "source": filename,
#                     "tenant_ID": tenant_ID,
#                     "page_content": vector["content"][:100],  # Store first 100 characters as a preview
#                 }

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector["id"], "values": vector["values"], "metadata": vector["metadata"]} for vector in vectors], namespace="")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

# # Main upload function using threading
# def upload_documents(directory, embeddings, index, tenant_ID):
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for filename in os.listdir(directory):
#             if filename.endswith(('.txt', '.pdf', '.docx')):  # Process only supported files
#                 futures.append(executor.submit(process_document, filename, directory, embeddings, index, tenant_ID))

#         # Wait for all futures to complete
#         for future in as_completed(futures):
#             future.result()  # This will also raise exceptions if any occurred during processing

#     print("All documents uploaded successfully!")

# # Example call to upload documents from the directory
# tenant_ID = "tenant1234"  # Example tenant ID
# directory = 'data'  # Specify the directory containing the documents
# upload_documents(directory, embeddings, index, tenant_ID)


In [36]:
# import os
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # Define constants
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to process each document
# def process_document(filename, directory, embeddings, index):
#     file_path = os.path.join(directory, filename)
#     document_text = read_document(file_path)
    
#     if document_text:
#         # Generate a random 14-digit document ID prefix
#         doc_id_prefix = generate_14_digit_random_id()

#         # Split the document into chunks
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         chunks = text_splitter.split_text(document_text)

#         # Generate embeddings for chunks in batches
#         vectors = []
#         for i, chunk in enumerate(chunks):
#             chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#             vectors.append({"id": chunk_id, "content": chunk})

#         # Generate embeddings for all chunks at once
#         if vectors:
#             content_list = [vector["content"] for vector in vectors]
#             embedded_vectors = embeddings.embed_documents(content_list)

#             # Prepare the vectors for upsert
#             for vector, embedding in zip(vectors, embedded_vectors):
#                 vector["values"] = embedding
#                 vector["metadata"] = {
#                     "source": filename,
#                     "page_content": vector["content"][:100],  # Store first 100 characters as a preview
#                 }

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector["id"], "values": vector["values"], "metadata": vector["metadata"]} for vector in vectors], namespace="")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

# # Main upload function using threading
# def upload_documents(directory, embeddings, index):
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for filename in os.listdir(directory):
#             if filename.endswith(('.txt', '.pdf', '.docx')):  # Process only supported files
#                 futures.append(executor.submit(process_document, filename, directory, embeddings, index))

#         # Wait for all futures to complete
#         for future in as_completed(futures):
#             future.result()  # This will also raise exceptions if any occurred during processing

#     print("All documents uploaded successfully!")

# # Example call to upload documents from the directory
# directory = 'data'  # Specify the directory containing the documents
# upload_documents(directory, embeddings, index)


In [60]:
#chunk_size = 500 
#chunk_overlap = 50  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

In [61]:
index_name = "test-2"
#vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
retriever = vectorstore.as_retriever()

In [62]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke("""From the values of X-ray and experimental den- sities, the percentage of porosity of the samples was calculated using the relation,.formula for percentage of porosity""")
print(format_docs(retrieved_docs))

In [64]:
query = "how to make an appointment"

In [65]:
for ids in index.list(namespace=""):
 query = index.query(
 id=ids[0], 
 top_k=1,
 include_values=True,
 include_metadata=True
 )
 print(query)

{'matches': [{'id': '0cd53c10-ab69-40ca-8c21-af16079c9e87',
              'metadata': {'source': 'data/world war.pdf',
                           'text': 'American Entry Into the War\n'
                                   '\n'
                                   'Through the Battle of Midway 1942, the US '
                                   'entered World War II. In this battle, US '
                                   'sea-based aircraft destroyed four Japanese '
                                   'carriers and a cruiser, marking the '
                                   'turning point in World War II. Also, the '
                                   'news of mass murders of Jewish people by '
                                   'the Nazis reached the Allies, and the US '
                                   'pledged to avenge these crimes.\n'
                                   '\n'
                                   'Reversal of German Fortunes\n'
                                   '\n'
     

In [66]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),

)
qa.invoke("how to make an appointment ")

{'query': 'how to make an appointment ',
 'result': 'To make a new appointment, follow these steps:\n\n1. Click the +Add button.\n2. Select the client from the drop-down menu.\n3. If the client is Checked In at the time, click on “Is Checked”.\n4. Proceed to select the appointment date and time.'}

In [41]:
template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents.
5. Answer for normal conversation questions like "Hi", "Hey", "Hello", "How are you?", and many others with the answer: "Hello, How can I assist you?".
6. If the question is on "summarize" or "summarization", then summarize the documents to (1/4)th the size of the original documents.

Question: {question}

Context: {context}

Answer:
"""

In [67]:
query1 = "how to make an appointment"

## With and Without Knowledge Base

In [68]:
# Send each query to the LLM twice, first with relevant knowledge from Pincone 
# and then without any additional knowledge.
print("Query 1\n")
print("Chat with knowledge:")
print(qa.invoke(query1).get("result"))
print("\nChat without knowledge:")
print(llm.invoke(query1).content)

Query 1

Chat with knowledge:
To make a new appointment, follow these steps:

1. Click the +Add button to start adding a new appointment.
2. Select the client from the drop-down menu.
3. If the client is checked in at the time, click on “Is Checked”.
4. Proceed to select the appointment date and time.

Chat without knowledge:
To make an appointment, follow these general steps:

1. **Identify the Purpose**: Determine the reason for the appointment, whether it's a medical visit, business meeting, or personal engagement.

2. **Choose the Service Provider**: Decide where or with whom you need to make the appointment. This could be a doctor's office, a business, or an individual.

3. **Contact Information**: Find the contact details of the service provider. This could be a phone number, email address, or an online booking system.

4. **Check Availability**: Look for available dates and times. This might involve checking an online calendar or calling to ask about open slots.

5. **Make the A

In [42]:
# custom_rag_template = PromptTemplate.from_template(template)

# # Create the parallel chain
# My_rag_chain = RunnableParallel(
#     {
#         "context": retriever | format_docs,
#         "question": RunnablePassthrough()
#     }
# ) | custom_rag_template | llm | StrOutputParser()

# ## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()

In [43]:
# keyword = "How to create a client?"
# template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

# Guidelines:
# 1. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
# 2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
# 3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
# 4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents.
# 5. Answer for normal conversation questions like "Hi", "Hey", "Hello", "How are you?", and many others with the answer: "Hello, How can I assist you?".
# 6. If the question is on "summarize" or "summarization", then summarize the documents to (1/4)th the size of the original documents.

# Question: {question}

# Context: {context}

# Answer:
# """

# prompt = template.format(question=keyword, context=format_docs(retrieved_docs))

# # Create the LLM
# llm = ChatOpenAI(
#     model="gpt-4o",
#     temperature=0.1,
# )

# # Set up the retrieval QA with custom response for missing answers
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=vectorstore.as_retriever(),
#     return_source_documents=True  # Ensures that source documents are returned
# )

# # Wrapper function to handle missing answers
# def get_answer(keyword):
#     response = qa.invoke(keyword)
    
#     # Check if the response contains valid data
#     result = response.get('result', '')
#     if "The answer is not specifically mentioned" in result or not result.strip():
#         return "Precise answer not found in documents, try another prompt."
#     else:
#         return result

# # Test the response with the keyword quer
# final_answer = get_answer("how to make a membership?")
# print(final_answer)

Found document with no `text` key. Skipping.


To create a membership program, you can follow these general steps:

1. **Define Your Purpose and Goals**: Determine why you want to create a membership program and what you hope to achieve. This could be to build a community, generate recurring revenue, or offer exclusive content or services.

2. **Identify Your Target Audience**: Understand who your potential members are, what they value, and what they are willing to pay for. This will help you tailor your offerings to meet their needs.

3. **Develop Your Offerings**: Decide what benefits, services, or products you will offer to members. This could include exclusive content, discounts, access to events, or other perks.

4. **Set Pricing and Tiers**: Determine how much you will charge for membership and whether you will offer different levels or tiers of membership with varying benefits.

5. **Choose a Platform**: Select a platform or software that will help you manage memberships, process payments, and deliver content. Options includ

In [44]:
pc.describe_index("test-2")


{
    "name": "test-2",
    "dimension": 1536,
    "metric": "dotproduct",
    "host": "test-2-5vwf04k.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [46]:
# # Function to delete all chunks of a document by prefix
# def delete_document_by_prefix(doc_id_prefix):
#     chunk_ids = list_document_chunks(doc_id_prefix)
#     if chunk_ids:
#         index.delete(ids=chunk_ids, namespace="")
#         print(f"Deleted {len(chunk_ids)} chunks for document: {doc_id_prefix}")
#     else:
#         print(f"No chunks found for document: {doc_id_prefix}")

# # Example usage
# # Upload documents from the directory
# upload_documents(directory, embeddings, index)

# # List all chunk IDs for a document
# doc_id_prefix = "cricket_rules"  # Replace with your actual document name
# chunks = list_document_chunks(doc_id_prefix)
# print(f"Chunks for document '{doc_id_prefix}': {chunks}")

# # Delete all chunks for a document
# delete_document_by_prefix(doc_id_prefix)

In [50]:
#index.delete(delete_all=True)


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 06 Oct 2024 19:30:31 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '34', 'x-pinecone-request-id': '7246155781317363175', 'x-envoy-upstream-service-time': '35', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}
